In [39]:
%pip install nltk
%pip install sklearn
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\giris\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\giris\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\giris\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [40]:
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer

In [41]:
# Load & check Data

colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    data = pd.read_csv(
        'drive/My Drive/COMP SCI 539/bbc-news-data.csv', delimiter='\t')
else:
    data = pd.read_csv('bbc-news-data.csv', delimiter='\t')

data.head()

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [42]:
# Data Preprocessing
# Find and remove nulls
print(data.isnull().sum())

# Data to lowercase
data["title"] = data["title"].str.lower()
data["content"] = data["content"].str.lower()
# Remove and replace contractions
# Find more contraction in text and add
contraction_dict = {"can't": "cannot", "didn't": "did not", "aren't": "are not", "she'd": "she would", "he'd": "he would", "they'd": "they would", "they've": "they have",
                    "shouldn't": "should not", "shouldn't've": "should not have", "she'll": "she will", "he'll": "he will", "they'll": "they will"
                    }


def contraction_replacer(text):
    for word in text.split():
        if word in contraction_dict:
            text = text.replace(word, contraction_dict[word])
    return text


data["title"] = data["title"].apply(contraction_replacer)
data["content"] = data["content"].apply(contraction_replacer)

# Remove punctuation and numbers
# Find more punctuation in text and add

numbers = '0123456789'


def punctuation_numbers_remover(text):
    for number in numbers:
        text = text.replace(number, '')
    return text


data["title"] = data["title"].apply(punctuation_numbers_remover)
data["content"] = data["content"].apply(punctuation_numbers_remover)

data.head()

# plot the distribution of the lengths of the sequences
content_lengths = [len(content.split()) for content in data["content"]]


# remove the outliers
MAX_CONTENT_LENGTH = 1000
MAX_TITLE_LENGTH = 10
# get the indices of the documents that have more than 1000 words
outliers = [idx for idx, length in enumerate(
    content_lengths) if length > MAX_CONTENT_LENGTH]

# remove the outliers from the data
data = data.drop(outliers, axis=0).reset_index(drop=True)

category    0
filename    0
title       0
content     0
dtype: int64


In [43]:
nltk.download('punkt')
nltk.download('stopwords')

# Sample text

predictions = []
stop_words = set(stopwords.words("english"))



for i in range (data["content"].shape[0]):
    print(f'Article {i}/{data["content"].shape[0]}', end='\r')
    text = data["content"][i]

    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Tokenize sentences into words, remove punctuation, and convert to lower case
    tokenizer = RegexpTokenizer(r'\w+')
    words_list = [tokenizer.tokenize(sentence.lower()) for sentence in sentences]

    # Remove stopwords
    filtered_words = [[word for word in words if word not in stop_words]
                      for words in words_list]

    # Combine preprocessed words into sentences again
    preprocessed_sentences = [' '.join(words) for words in filtered_words]

    vectorizer = TfidfVectorizer()

    # Fit the vectorizer and transform sentences into TF-IDF vectors
    tfidf_vectors = vectorizer.fit_transform(preprocessed_sentences)

    # Create a graph to represent the sentences and their connections
    graph = nx.Graph()

    # Add nodes to the graph (each sentence is a node)
    graph.add_nodes_from(sentences)

    # Calculate cosine similarity between sentences and create edges between them based on similarity
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                similarity = tfidf_vectors[i].dot(tfidf_vectors[j].T).toarray()[
                    0][0]  # Compute cosine similarity
                graph.add_edge(sentences[i], sentences[j], weight=similarity)

    # Apply the PageRank algorithm to rank the sentences
    scores = nx.pagerank(graph)

    # Sort the sentences based on their scores
    ranked_sentences = sorted(((scores[sentence], sentence)
                              for sentence in sentences), reverse=True)

    # Number of sentences to include in the summary
    num_sentences = 1  # Change this as needed

    # Extract top-ranked sentences to create the summary
    summary_sentences = sorted(ranked_sentences[:num_sentences])

    # Generate the final summary
    summary = " ".join(sentence[1] for sentence in summary_sentences)

    # print("Original Text:\n", text)
    # print("\nSummary:\n", summary)
    predictions.append(summary)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
from rouge_score import rouge_scorer
# Evaluation


rouge1 = 0.0
rougeL = 0.0

def calculate_rouge(reference, generated):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return scores

for i in range(len(predictions)):
    sc = calculate_rouge(data["title"][i], predictions[i])
    rouge1 += sc["rouge1"].fmeasure
    rougeL += sc["rougeL"].fmeasure

rouge1 /= len(predictions)
rougeL /= len(predictions)

print("ROUGE-1: ", rouge1)
print("ROUGE-L: ", rougeL)

ROUGE-1:  0.13455555123929014
ROUGE-L:  0.11793104975838294
